## Data Combination Process

### Step 1: Data Extraction
Before we can combine the data, we need to first extract and prepare our raw data files. This is explained in detail in another notebook called **Data Extraction**. In that notebook, we will:

1. Download and organize the raw CSV files into the `data/raw` directory.
2. Ensure that all the CSV files are correctly formatted and contain the required data.

Make sure to run the steps in the **Data Extraction** notebook before proceeding with the next step.

### Step 2: Running the Data Combination Script
Once the raw data files are available in the `data/raw` folder, you can use the following Python script to combine them into one single CSV file. This script:

1. Reads all CSV files in the `data/raw` folder.
2. Combines them into a single DataFrame.
3. Saves the combined DataFrame as `combined_ais_data.csv` in a new folder called `processed`.

If the `processed` folder does not exist, the script will create it automatically.

Run the script to generate the combined data that can be used for further analysis or processing.

### Example Workflow:
1. **Run the Data Extraction notebook** to prepare the `data/raw` folder.
2. **Run the Python script** to combine the data into `combined_ais_data.csv` in the `processed` folder.


## Python Script to Combine the Data

The Python script provided combines multiple CSV files from the `data/raw` folder into one consolidated CSV file:

In [1]:
import os
import pandas as pd
import glob

# Define the project root directory name
project_root = 'DSCI-532_2025_5_vessel-vision'


current_directory = os.getcwd()


while project_root not in os.path.basename(current_directory) and current_directory != os.path.dirname(current_directory):
    current_directory = os.path.dirname(current_directory)

if project_root not in os.path.basename(current_directory):
    raise ValueError(f"Project root '{project_root}' not found.")


os.chdir(current_directory)
print(f"Current working directory: {os.getcwd()}")  


raw_data_folder = os.path.join('data', 'raw')
processed_folder_path = os.path.join('data', 'processed') 
raw_data_path = os.path.join(raw_data_folder, '*.csv')
combined_csv_path = os.path.join(processed_folder_path, 'combined_ais_data.csv')

# Ensure the raw data folder exists
if not os.path.exists(raw_data_folder):
    raise FileNotFoundError(
        f"Data folder not found: {raw_data_folder}\n"
        "Please execute the data extraction file first to generate the required data."
    )

# Get all CSV files in the 'data/raw' folder
csv_files = glob.glob(raw_data_path)

# Check if any CSV files are found
if not csv_files:
    raise ValueError(
        "No CSV files found in 'data/raw/'.\n"
        "Please execute the data extraction file first to generate the required data."
    )

# Ensure the processed folder exists
os.makedirs(processed_folder_path, exist_ok=True)

# Read and combine all CSV files
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to 'data/processed'
combined_df.to_csv(combined_csv_path, index=False)

print(f'✅ Combined CSV file saved at: {combined_csv_path}')


Current working directory: C:\Users\Azin\Desktop\Azin files\Azin's Document\UBC\block 5\532_viz-2\DSCI-532_2025_5_vessel-vision
✅ Combined CSV file saved at: data\processed\combined_ais_data.csv


## Filtering AIS Data for the West Coast of North America

To only include the **West Coast of North America**, you need to filter based on **longitude and latitude**.

### Steps:
1. **Identify geographic boundaries** of the West Coast of North America.
   - Covers **Canada, USA, and Mexico’s Pacific coasts**.
   
2. **Approximate latitude/longitude range**:
   - **Latitude**: 20°N to 60°N  
   - **Longitude**: -140°W to -110°W
3. **Remove unwanted columns**:
   - In the dataset, the following columns were removed: `COG`, `Heading`, `IMO`, `CallSign`, `Length`, `Width`, `Draft`, `Cargo`, `TransceiverClass`. These columns were deemed unnecessary for the analysis, so they were dropped to clean the data and focus on relevant information.


In [2]:
import os
import pandas as pd
import math

# Define the project root directory name
project_root = 'DSCI-532_2025_5_vessel-vision'

# Get the absolute path of the current working directory
current_directory = os.getcwd()

# Find the root directory path
while project_root not in os.path.basename(current_directory) and current_directory != os.path.dirname(current_directory):
    current_directory = os.path.dirname(current_directory)

if project_root not in os.path.basename(current_directory):
    raise ValueError(f"Project root '{project_root}' not found.")

# Change to the root directory
os.chdir(current_directory)
print(f"Current working directory: {os.getcwd()}")  # Debugging statement

# Define paths
preprocessed_folder = os.path.join('data', 'processed')
combined_csv_path = os.path.join(preprocessed_folder, 'combined_ais_data.csv')
filtered_csv_path = os.path.join(preprocessed_folder, 'ais_west_coast.csv')

# Ensure the preprocessed data folder exists
if not os.path.exists(preprocessed_folder):
    raise FileNotFoundError(
        f"Preprocessed data folder not found: {preprocessed_folder}\n"
        "Please execute the data extraction and combination steps first."
    )

# Load the combined AIS dataset
df = pd.read_csv(combined_csv_path)

# Define latitude and longitude boundaries for the West Coast
lat_min, lat_max = 20, 60   # Covers from Mexico to Alaska
lon_min, lon_max = -140, -110  # Covers the Pacific coast range

# Filter data for West Coast
west_coast_df = df[
    (df['LAT'].between(lat_min, lat_max)) & 
    (df['LON'].between(lon_min, lon_max))
]

# Define a sample list of port coordinates (latitude, longitude) and their names
ports = [
    {"port": "Port of Los Angeles", "lat": 33.74, "lon": -118.26},
    {"port": "Port of Seattle", "lat": 47.60, "lon": -122.33},
    {"port": "Port of San Francisco", "lat": 37.78, "lon": -122.42},
    {"port": "Port of Vancouver", "lat": 49.28, "lon": -123.12},
    {"port": "Port of Manzanillo", "lat": 19.05, "lon": -104.33},  # Mexico
    {"port": "Port of Ensenada", "lat": 31.86, "lon": -116.60},  # Mexico
    {"port": "Port of Mazatlán", "lat": 23.25, "lon": -106.41},  # Mexico
    {"port": "Port of Lázaro Cárdenas", "lat": 18.12, "lon": -102.18},  # Mexico
    {"port": "Port of Acapulco", "lat": 16.86, "lon": -99.88},  # Mexico
    {"port": "Port of Long Beach", "lat": 33.75, "lon": -118.20},  # U.S.
    {"port": "Port of Oakland", "lat": 37.80, "lon": -122.27},  # U.S.
    {"port": "Port of San Diego", "lat": 32.72, "lon": -117.17},  # U.S.
    {"port": "Port of Tacoma", "lat": 47.26, "lon": -122.43},  # U.S.
]

# Function to calculate the distance between two points using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Earth's radius in kilometers
    radius = 6371.0
    
    # Distance in kilometers
    return radius * c

# Function to find the nearest port based on latitude and longitude
def find_nearest_port(lat, lon, ports):
    nearest_port = None
    min_distance = float('inf')
    
    for port in ports:
        port_coords = (port["lat"], port["lon"])
        distance = haversine(lat, lon, port_coords[0], port_coords[1])  # Distance in kilometers
        
        if distance < min_distance:
            min_distance = distance
            nearest_port = port["port"]
    
    return nearest_port

# Add vessel type name column
west_coast_df['Vessel Type Name'] = west_coast_df['VesselType'].apply(
    lambda x: 'Passenger' if 60 <= x <= 69 else ('Cargo' if 70 <= x <= 79 else None)
)

# Filter only Passenger and Cargo vessels
filtered_west_coast_df = west_coast_df[west_coast_df['Vessel Type Name'].notna()]

# Find the nearest port for each vessel based on latitude and longitude
filtered_west_coast_df['Nearest Port'] = filtered_west_coast_df.apply(
    lambda row: find_nearest_port(row['LAT'], row['LON'], ports), axis=1
)

# Remove specified columns
columns_to_remove = ['COG', 'Heading', 'IMO', 'CallSign', 'Length', 'Width', 'Draft', 'Cargo', 'TransceiverClass']
filtered_west_coast_df.drop(columns=columns_to_remove, inplace=True, errors='ignore')

# Save the filtered dataset
filtered_west_coast_df.to_csv(filtered_csv_path, index=False)

print(f'✅ Filtered dataset saved at: {filtered_csv_path} with {len(filtered_west_coast_df)} records.')


Current working directory: C:\Users\Azin\Desktop\Azin files\Azin's Document\UBC\block 5\532_viz-2\DSCI-532_2025_5_vessel-vision


C:\Users\Azin\AppData\Local\Temp\ipykernel_43496\1215247604.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  west_coast_df['Vessel Type Name'] = west_coast_df['VesselType'].apply(
C:\Users\Azin\AppData\Local\Temp\ipykernel_43496\1215247604.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_west_coast_df['Nearest Port'] = filtered_west_coast_df.apply(
C:\Users\Azin\AppData\Local\Temp\ipykernel_43496\1215247604.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

✅ Filtered dataset saved at: data\processed\ais_west_coast.csv with 579340 records.


## Split filtered data into multiple CSV files
This script splits a large dataset (ais_west_coast.csv) into smaller, approximately 50MB-sized CSV files and stores them in a split-data folder. Before processing the data, the script ensures that it is running from the root directory of the project (DSCI-532_2025_5_vessel-vision). The dataset is read in chunks of 100,000 rows at a time, and the size of each chunk is calculated to ensure it doesn’t exceed 50MB. Once a chunk reaches the target size, it is written to a CSV file with sequential names like ais_chunk_0.csv, ais_chunk_1.csv, etc. If there is leftover data after reading the entire dataset, it is saved to a final chunk file.

In [3]:
import pandas as pd
import os
import shutil

# Define parameters
data_path = "data/processed/ais_west_coast.csv"
output_folder = "data/split-data/"
project_root = "DSCI-532_2025_5_vessel-vision"
chunk_size_in_mb = 100  # Target chunk size in MB (maximum size allowed by GitHub)

# Get the current directory
current_directory = os.getcwd()

# Define the correct project root directory
root_directory = r"c:\Users\Azin\Desktop\Azin files\Azin's Document\UBC\block 5\532_viz-2\DSCI-532_2025_5_vessel-vision"

# Check if we are in the root directory
if current_directory != root_directory:
    # If not in root, navigate to the root directory
    if os.path.exists(root_directory):
        os.chdir(root_directory)
        print(f"Changed directory to project root: {root_directory}")
    else:
        print(f"Error: The root directory '{root_directory}' does not exist.")
else:
    print(f"Already in the root directory: {current_directory}")

# Remove the split-data folder if it exists and recreate it
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove the existing folder
    print(f"Removed existing folder: {output_folder}")

# Create the split-data folder again
os.makedirs(output_folder, exist_ok=True)
print(f"Created new folder: {output_folder}")

# Get the size of the file in bytes
file_size = os.path.getsize(data_path)  # In bytes
target_chunk_size = chunk_size_in_mb * 1024 * 1024  # Convert to bytes

# Read the dataset in chunks
chunk_index = 0
for chunk in pd.read_csv(data_path, chunksize=1000000):  # Read in manageable chunks
    # Calculate approximate size of the chunk (in bytes)
    chunk_size = chunk.memory_usage(deep=True).sum()

    # If the chunk size exceeds the target size, split the chunk
    while chunk_size > target_chunk_size:
        # Split into smaller chunks (split by rows)
        half_chunk = chunk.iloc[:len(chunk) // 2]
        chunk = chunk.iloc[len(chunk) // 2:]

        # Save the first half
        half_chunk_filename = f"ais_chunk_{chunk_index}_part1.csv"
        half_chunk.to_csv(os.path.join(output_folder, half_chunk_filename), index=False)
        print(f"Written {half_chunk_filename}")
        chunk_index += 1
        
        # Recalculate chunk size for the remaining part
        chunk_size = chunk.memory_usage(deep=True).sum()

    # Save the remaining chunk
    chunk_filename = f"ais_chunk_{chunk_index}.csv"
    chunk.to_csv(os.path.join(output_folder, chunk_filename), index=False)
    print(f"Written {chunk_filename}")
    chunk_index += 1

print("Data splitting completed.")




Changed directory to project root: c:\Users\Azin\Desktop\Azin files\Azin's Document\UBC\block 5\532_viz-2\DSCI-532_2025_5_vessel-vision
Removed existing folder: data/split-data/
Created new folder: data/split-data/
Written ais_chunk_0_part1.csv
Written ais_chunk_1.csv
Data splitting completed.
